## Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic;
- для источников paid и email из России - ставим ad;
- для источников paid и email не из России - ставим other;
- все остальные варианты берем из traffic_source без изменений.

In [1]:
import pandas as pd

**Вариант 1** (Странный и немного медленный, но вроде как универсальный и рабочий. Можно различные фильтры задавать через словарь filters_dict.)

In [2]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [3]:
regions = set(log.region)

filters_dict = {
    'organic':{'region':regions,'traffic_source':{'yandex', 'google'}},
    'ad':{'region':{'Russia'},'traffic_source':{'paid', 'email'}},
    'other':{'region':regions.difference({'Russia'}),'traffic_source':{'paid', 'email'}}
}

def source_type_classificator(row):
    for source_type, filters_subdict in filters_dict.items():
        for filter_key,filter_values_list in filters_subdict.items():
            for filter_value in filter_values_list:
                if filter_value in row[filter_key]:
                    condition_performed = True
                    break
                else:
                    condition_performed = False
            if condition_performed == False:
                break
        if condition_performed == True:
            return source_type
    return row['traffic_source']

In [4]:
%%time
log['source_type'] = log.apply(source_type_classificator,axis=1)
log.head(20)

Wall time: 1.2 s


,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,ad
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,direct
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,direct
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,ad


**Вариант 2** (Вариант довольно быстрый, но топорный. Наверное должен быть какой-то третий вариант.)

In [5]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [6]:
%%time
log.loc[log.traffic_source.isin(['yandex','google']),'traffic_type'] = 'organic'
log.loc[log.traffic_source.isin(['paid','email'])&log.region.isin(['Russia']),'traffic_type'] = 'ad'
log.loc[log.traffic_source.isin(['paid','email'])&~log.region.isin(['Russia']),'traffic_type'] = 'other'
log.traffic_type = log.traffic_type.fillna(log.traffic_source)
log.head(20)

Wall time: 12 ms


,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,ad
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,direct
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,direct
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,ad


## Задание 2
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:
- Прочитайте содержимое файла с датафрейм;
- Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном.

In [7]:
log = pd.read_table('URLs.txt')
log.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [8]:
log.loc[log.url.str.contains(r'/\d{8}-.'),'news'] = True
log.news = log.fillna(False).news
log.head()

,url,news
0,/world/,False
1,/latest/,False
2,/?updated=top,False
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...,True
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...,True


## Задание 3
В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [9]:
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})
data.head()

,keyword,shows
0,курс гривны к рублю,125076
1,доллары в рубли,114173
2,100 долларов в рублях,97534
3,курс рубля,53546


In [10]:
from pymystem3 import Mystem

In [11]:
%%time
data['lemmas'] = data.keyword.apply(lambda search: Mystem().lemmatize(search)) 
data.head()

Wall time: 3.18 s


,keyword,shows,lemmas
0,курс гривны к рублю,125076,"[курс, , гривна, , к, , рубль, \n]"
1,доллары в рубли,114173,"[доллар, , в, , рубль, \n]"
2,100 долларов в рублях,97534,"[100, , доллар, , в, , рубль, \n]"
3,курс рубля,53546,"[курс, , рубль, \n]"
